# HW 3

Name: Aryan S Patel


EUID: asp0144


ID: 11290269

Additional libraries were used to plot the graphs, enhancing our understanding of the data in question 3.   
Library used for plotting is called plotly and can be downloaded using this command: `pip3 install plotly`

##### Getting Dataset

In [7]:
!!/usr/bin/curl --output test.dat https://raw.githubusercontent.com/huangyanann/CSCE5218/main/test_small.txt
!!/usr/bin/curl --output train.dat https://raw.githubusercontent.com/huangyanann/CSCE5218/main/train.txt

['  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current',
 '                                 Dload  Upload   Total   Spent    Left  Speed',
 '',
 '  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0',
 '100 11645  100 11645    0     0   217k      0 --:--:-- --:--:-- --:--:--  218k']

##### Taking a peek

In [1]:
!head train.dat
!head test.dat

A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13	
1	1	0	0	0	0	0	0	1	1	0	0	1	0
0	0	1	1	0	1	1	0	0	0	0	0	1	0
0	1	0	1	1	0	1	0	1	1	1	0	1	1
0	0	1	0	0	1	0	1	0	1	1	1	1	0
0	1	0	0	0	0	0	1	1	1	1	1	1	0
0	1	1	1	0	0	0	1	0	1	1	0	1	1
0	1	1	0	0	0	1	0	0	0	0	0	1	0
0	0	0	1	1	0	1	1	1	0	0	0	1	0
0	0	0	0	0	0	1	0	1	0	1	0	1	0
X1	X2	X3
1	1	1	1
0	0	1	1
0	1	1	0
0	1	1	0
0	1	1	0
0	1	1	0
0	1	1	0
0	1	1	0
1	1	1	1


##### Building the Perceptron Model

In [2]:
import math
import itertools
import re


# Corpus reader, all columns but the last one are coordinates;
#   the last column is the label
def read_data(file_name):
    f = open(file_name, 'r')

    data = []
    # Discard header line
    f.readline()
    for instance in f.readlines():
        if not re.search('\t', instance): continue
        instance = list(map(int, instance.strip().split('\t')))
        # Add a dummy input so that w0 becomes the bias
        instance = [-1] + instance
        data += [instance]
    return data


def dot_product(array1, array2):
    # Calculating and returning dot products between two arrays
    return sum(a * b for a, b in zip(array1, array2))


def sigmoid(x):
    # Calculating and returining sigmoid function of input variable
    return 1 / (1 + math.exp(-x))

# The output of the model, which for the perceptron is 
# the sigmoid function applied to the dot product of 
# the instance and the weights
def output(weight, instance):
    # returning output
    return sigmoid(dot_product(instance, weight))

# Predict the label of an instance; this is the definition of the perceptron
# you should output 1 if the output is >= 0.5 else output 0
def predict(weights, instance):
    # returning prediction
    return 1 if output(weights, instance) >= 0.5 else 0


# Accuracy = percent of correct predictions
def get_accuracy(weights, instances):
    # You do not to write code like this, but get used to it
    correct = sum([1 if predict(weights, instance) == instance[-1] else 0
                   for instance in instances])
    return correct * 100 / len(instances)


# Train a perceptron with instances and hyperparameters:
#       lr (learning rate) 
#       epochs
# The implementation comes from the definition of the perceptron
#
# Training consists on fitting the parameters which are the weights
# that's the only thing training is responsible to fit
# (recall that w0 is the bias, and w1..wn are the weights for each coordinate)
#
# Hyperparameters (lr and epochs) are given to the training algorithm
# We are updating weights in the opposite direction of the gradient of the error,
# so with a "decent" lr we are guaranteed to reduce the error after each iteration.
def train_perceptron(instances, lr, epochs):

    #TODO: name this step
    # Initializing weights
    weights = [0] * (len(instances[0])-1)

    for _ in range(epochs):
        for instance in instances:
            #TODO: name these steps
            # Calculating predicted value
            in_value = dot_product(weights, instance)
            output = sigmoid(in_value)
            error = instance[-1] - output
            #TODO: name these steps
            # Update weights using gradient descent
            for i in range(0, len(weights)):
                weights[i] += lr * error * output * (1-output) * instance[i]

    return weights

##### Running it

In [3]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
lr = 0.005
epochs = 5
weights = train_perceptron(instances_tr, lr, epochs)
accuracy = get_accuracy(weights, instances_te)
print(f"#tr: {len(instances_tr):3}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
      f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#tr: 400, epochs:   5, learning rate: 0.005; Accuracy (test, 14 instances): 71.4


### Questions

##### Question 1

We are not assigning the output directly to the prediction because the statement `output = predict(weights, instance)` only indicates whether the output is 1 or 0. It does not provide information on how far off we are from making a correct prediction. Without knowing this distance, we cannot effectively adjust the weights of the parameters to improve our chances of getting the correct answer.

##### Question 2

In [4]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]     # number of epochs
lr_array = [0.005, 0.01, 0.05]        # learning rate

result = []

for lr in lr_array:
  for tr_size in tr_percent:
    for epochs in num_epochs:
      size =  round(len(instances_tr)*tr_size/100)
      pre_instances = instances_tr[0:size]
      weights = train_perceptron(pre_instances, lr, epochs)
      accuracy = get_accuracy(weights, instances_te)
      print(f"#tr: {len(pre_instances):0}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
          f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")
      temp = [len(pre_instances), epochs, lr, accuracy]
      result.append(temp)

#tr: 20, epochs:   5, learning rate: 0.005; Accuracy (test, 14 instances): 71.4
#tr: 20, epochs:  10, learning rate: 0.005; Accuracy (test, 14 instances): 71.4
#tr: 20, epochs:  20, learning rate: 0.005; Accuracy (test, 14 instances): 71.4
#tr: 20, epochs:  50, learning rate: 0.005; Accuracy (test, 14 instances): 71.4
#tr: 20, epochs: 100, learning rate: 0.005; Accuracy (test, 14 instances): 85.7
#tr: 40, epochs:   5, learning rate: 0.005; Accuracy (test, 14 instances): 71.4
#tr: 40, epochs:  10, learning rate: 0.005; Accuracy (test, 14 instances): 71.4
#tr: 40, epochs:  20, learning rate: 0.005; Accuracy (test, 14 instances): 71.4
#tr: 40, epochs:  50, learning rate: 0.005; Accuracy (test, 14 instances): 71.4
#tr: 40, epochs: 100, learning rate: 0.005; Accuracy (test, 14 instances): 71.4
#tr: 100, epochs:   5, learning rate: 0.005; Accuracy (test, 14 instances): 71.4
#tr: 100, epochs:  10, learning rate: 0.005; Accuracy (test, 14 instances): 71.4
#tr: 100, epochs:  20, learning rate: 

##### Question 3

Code for plotting graph to better answers the questions

In [5]:
highest_accuracy = []
high_acc = 0
i = 0
while i < len(result):
    if result[i][3] > high_acc:
        highest_accuracy.clear()
        high_acc = result[i][3]
        highest_accuracy.append(result[i])
    elif result[i][3] == high_acc:
        highest_accuracy.append(result[i])
    i += 1

lr_0005 = []
lr_001 = []
lr_005 = []

i = 0
while i < len(highest_accuracy):
    if highest_accuracy[i][2] == 0.005:
        lr_0005.append(highest_accuracy[i])
    elif highest_accuracy[i][2] == 0.01:
        lr_001.append(highest_accuracy[i])
    elif highest_accuracy[i][2] == 0.05:
        lr_005.append(highest_accuracy[i])
    i += 1

In [6]:
import plotly.graph_objects as go

def plotting(lis):
    i = 0
    x = []
    y = []
    while i < len(lis):
        x.append(lis[i][0])
        y.append(lis[i][1])
        i += 1
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode='markers'))
    
    fig.update_layout(
        title=f'{lis[0][2]} Learning rate combination with Highest Accuracy',
        xaxis_title='Training data',
        yaxis_title='Epochs'
    )
    
    fig.show()

print(f'Highest accuracy: {lr_0005[0][-1]:.2f}')
plotting(lr_0005)
plotting(lr_001)
plotting(lr_005)

Highest accuracy: 85.71


##### A

No, you don't need to train with all the data to achieve the highest accuracy. In fact, using more data can lead to lower accuracy. Based on the graphs provided, it appears that 200 samples which is half of the training data seems to be the ideal size for achieving the highest possible accuracy from the model.

##### B

Adding more data can lead to overfitting or underfitting, especially if the learning rate is too high or the number of epochs is too large. Instead of learning, the model will start to memorize the data, decreasing accuracy.

##### C

Getting higher accuracy is possible by adding additional hyperparameters like beta decay.

##### D

No, it isn't worth training for more epochs. As shown in the graphs above, you can achieve the same level of accuracy more consistently with fewer epochs rather than more.